# 

# Notebook Summary 


### Quickstart

  1. Import etiq library - for install please check our docs (https://docs.etiq.ai/) 

  2. Login to the dashboard - this way you can send the results to your dashboard instance (Etiq AWS instance if you use the SaaS version). To deploy on your own cloud instance, get in touch (info@etiq.ai)

  3. Create or open a project 
  
### Example dataset & model


  4. Example model: using Adult dataset, predicts income above/below 50K 
  
### Create a snapshot 

  5. Log dataset & model 
  
  6. Load a config file
  
  6. Create a snapshot 
  
  
### Scan your snapshot 
  
  7. Run a scan 
  
  8. Retrieve results
  
  
### Example other scans 

# 

# Quickstart 

In [1]:
import etiq


Thanks for trying out the ETIQ.AI toolkit!

This is a trial version, you have 14 days remaining in your trial period.
Please consider purchasing the full version to continue enjoying all the features of our library.

Visit our getting started documentation at https://docs.etiq.ai/

Visit our Slack channel at https://etiqcore.slack.com/ for support or feedback.
Help improve our product: Call `etiq.enable_telemetry()` to provide
anonymous library usage statistics.
        


In [2]:
from etiq import login as etiq_login
etiq_login("https://dashboard.etiq.ai/", "<token>")

'Connection successful. Projects and pipelines will be displayed in the dashboard. 😀'

In [3]:
# Can enumerate all available projects
all_projects = etiq.projects.get_all_projects()
print(all_projects)

[<ETIQ:Project [fkjdT9NzKEKqDu2yX5uzC9] "Demo Already Trained">]


In [4]:
# Can get/create a single named project
project = etiq.projects.open(name="Demo Already Trained")

# 

# Example dataset and model

To illustrate some of the library's features, we build a model that predicts whether an applicant makes over or under 50K using the Adult dataset from https://archive.ics.uci.edu/ml/datasets/adult.

1. Import the dataset

2. Pre-process the dataset 

3. Build the model 

4. Log the dataset and already built model to Etiq


In this case we encode prior to splitting into test/train/validate because we know in advance the categories people fall into for this dataset. This means that in production we won't run into new categories that will fall into a bucket not included in this dataset, This allows us to encode prior to splitting into train/test/validation. 

However if this is not the case for your use case, you should NOT encode prior to splitting your sample, as this might lead to LEAKAGE. 

Encoding categorical values itself is problematic as it assigns a numerical ranking to categorical variables. For best practice encoding use one hot encoding. As we limit the free library functionality to 15 features, we will not do one-hot encoding for the purposes of this example. 

Remember: This is an example only. The use case for the majority of scans in Etiq is that you log the model to Etiq once you have the sample that you'll be training on. Usually this sample will have numeric features only as otherwise you will not be able to use it in with the majority of supported libraries training methods. 

In [5]:
import pandas as pd
import numpy as np
import json
from xgboost.sklearn import XGBClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore')

# Loading a dataset. We're using the adult dataset
data = etiq.utils.load_sample("adultdataset.csv")
data.replace("?", np.nan)
data.dropna(inplace=True)
data.head()

# use a LabelEncoder to transform categorical variables
cont_vars = ["age", "educational-num", "fnlwgt", "capital-gain", "capital-loss", "hours-per-week"]
cat_vars = sorted(list(set(data.columns.values) - set(cont_vars)))

label_encoders = {}
data_encoded = pd.DataFrame()
for i in cat_vars:
    label = LabelEncoder()
    data_encoded[i] = label.fit_transform(data[i])
    label_encoders[i] = label

data_encoded.set_index(data.index, inplace=True)
data_encoded = pd.concat([data.loc[:, cont_vars], data_encoded], axis=1).copy()



In [6]:
# prepare the training/testing/validation datasets

# separate into train/validate/test dataset of sizes 80%/10%/10% as percetages of the initial data
data_remaining, test = train_test_split(data_encoded, test_size=0.1)
train, valid = train_test_split(data_remaining, test_size=0.1112)

# because we don't want to train on protected attributes or labels to be predicted, 
# let's remove these columns from the training dataset
protected_train = train['gender'].copy() # gender is a protected attribute
y_train = train['income'].copy() # labels we're going to train the model to predict
x_train = train.drop(columns=['gender','income'])
protected_valid = valid['gender'].copy() 
y_valid = valid['income'].copy() 
x_valid = valid.drop(columns=['gender','income'])
protected_test = test['gender'].copy() 
y_test = test['income'].copy()
x_test = test.drop(columns=['gender','income'])

In [7]:
# train a XGBoost model to predict 'income'

standard_model = XGBClassifier(eval_metric='logloss', random_state=4)    
model_fit = standard_model.fit(x_train, y_train)

In [8]:
y_train_pred = standard_model.predict(x_train)
y_valid_pred = standard_model.predict(x_valid)
print('Model accuracy on the training dataset :', 
      round(100 * accuracy_score(y_train, y_train_pred),2),'%') # round the score to 2 digits  

print('Model accuracy on the validation dataset :', 
      round(100 * accuracy_score(y_valid, y_valid_pred),2),'%')

Model accuracy on the training dataset : 90.1 %
Model accuracy on the validation dataset : 87.69 %


# 

# Log dataset & model snapshot to Etiq 

This is an example of how you log the already trained model to Etiq.

When you log the dataset make sure you log a new sample or a test sample, not the sample you trained you model on. 

When you log the config, make sure the % for train and valid splits are 0. 

If you are planning to use this in production, not just pre-production get in touch with us. We will release integration demos shortly.


## Loading the config file 

The config is where you can set-up the scans you want, and the thresholds outside of which Etiq finds a problem. In the config you input relevant parameters (e.g. for bias/fairness scans you'll have to tell Etiq which feature is a demographic and which value represents a protected group). 
For more details on the config just check the documentation. You can upload these config files from wherever you want. We provide examples in the Demo repo with each notebook.   

The config gets stored in the database so you can have a log and version control. 

When you log the config for an already trained model, please log 



In [9]:

etiq.load_config("./config_already_trained.json")


{'dataset': {'label': 'income',
  'bias_params': {'protected': 'gender',
   'privileged': 1,
   'unprivileged': 0,
   'positive_outcome_label': 1,
   'negative_outcome_label': 0},
  'train_valid_test_splits': [0.0, 1.0, 0.0],
  'remove_protected_from_features': True},
 'scan_accuracy_metrics': {'thresholds': {'accuracy': [0.8, 1.0],
   'true_pos_rate': [0.6, 1.0],
   'true_neg_rate': [0.6, 1.0]}},
 'scan_bias_metrics': {'thresholds': {'equal_opportunity': [0.0, 0.2],
   'demographic_parity': [0.0, 0.2],
   'equal_odds_tnr': [0.0, 0.2],
   'individual_fairness': [0.0, 0.8],
   'equal_odds_tpr': [0.0, 0.2]}},
 'scan_leakage': {'leakage_threshold': 0.85}}

## Log the dataset and model

In [10]:
from etiq import Model


#log your dataset

dataset = etiq.BiasDatasetBuilder.dataset(test)
bias_params = etiq.BiasDatasetBuilder.bias_params()
#Log your already trained model

model = Model(model_architecture=standard_model, model_fitted=model_fit)



## Creating a snapshot

In [11]:
# Creating a snapshot
snapshot = project.snapshots.create(name="Test Snapshot",
                                    dataset=dataset,
                                    model=model,
                                    bias_params=bias_params)



INFO:etiq.charting:Created histogram summary of data (14 fields)


# 

# Run scans

You can run multiple scans for snapshot. 

## Bias Metrics 

In [12]:
#scan_bias_metrics

(segments, issues, issue_summary) = snapshot.scan_bias_metrics()

Exclude Features = None
INFO:etiq.pipeline.BiasMetricsIssuePipeline0374:Starting pipeline
INFO:etiq.pipeline.BiasMetricsIssuePipeline0374:Computed bias metrics for the dataset
INFO:etiq.pipeline.BiasMetricsIssuePipeline0374:Threshold issues ['demographic_parity_above_threshold']
INFO:etiq.pipeline.BiasMetricsIssuePipeline0374:Min threshold = 0.0, Max threshold = 0.2
INFO:etiq.pipeline.BiasMetricsIssuePipeline0374:Threshold issues ['equal_odds_tpr_above_threshold']
INFO:etiq.pipeline.BiasMetricsIssuePipeline0374:Min threshold = 0.0, Max threshold = 0.2
INFO:etiq.pipeline.BiasMetricsIssuePipeline0374:Threshold issues ['equal_odds_tnr_above_threshold']
INFO:etiq.pipeline.BiasMetricsIssuePipeline0374:Min threshold = 0.0, Max threshold = 0.2
INFO:etiq.pipeline.BiasMetricsIssuePipeline0374:Threshold issues ['equal_opportunity_above_threshold']
INFO:etiq.pipeline.BiasMetricsIssuePipeline0374:Min threshold = 0.0, Max threshold = 0.2
INFO:etiq.pipeline.BiasMetricsIssuePipeline0374:Threshold iss

In [13]:
issues

""


In [14]:
issue_summary

,name,metric,measure,features,segments,total_issues_tested,issues_found,threshold
0,demographic_parity_above_threshold,<function demographic_parity at 0x7f192e1e4d30>,None,{},{},1,0,"[0.0, 0.2]"
1,equal_odds_tpr_above_threshold,<function equal_odds_tpr at 0x7f192e1e4e50>,None,{},{},1,0,"[0.0, 0.2]"
2,equal_odds_tnr_above_threshold,<function equal_odds_tnr at 0x7f192e1e4f70>,None,{},{},1,0,"[0.0, 0.2]"
3,equal_opportunity_above_threshold,<function equal_opportunity at 0x7f192e1ee0d0>,None,{},{},1,0,"[0.0, 0.2]"
4,individual_fairness_above_threshold,<function individual_fairness at 0x7f192e1ee3a0>,None,{},{},1,0,"[0.0, 0.8]"


## Accuracy metrics scan

In [15]:
(segments, issues, issue_summary) = snapshot.scan_accuracy_metrics()

INFO:etiq.pipeline.AccuracyMetricsIssuePipeline0099:Starting pipeline
INFO:etiq.pipeline.AccuracyMetricsIssuePipeline0099:Computed acurracy metrics for the dataset {'accuracy': 0.87, 'true_pos_rate': 0.6434108527131783, 'true_neg_rate': 0.9374328678839957}
INFO:etiq.pipeline.AccuracyMetricsIssuePipeline0099:Threshold issues ['accuracy_below_threshold']
INFO:etiq.pipeline.AccuracyMetricsIssuePipeline0099:Min threshold = 0.8, Max threshold = 1.0
INFO:etiq.pipeline.AccuracyMetricsIssuePipeline0099:Threshold issues ['true_pos_rate_below_threshold']
INFO:etiq.pipeline.AccuracyMetricsIssuePipeline0099:Min threshold = 0.6, Max threshold = 1.0
INFO:etiq.pipeline.AccuracyMetricsIssuePipeline0099:Threshold issues ['true_neg_rate_below_threshold']
INFO:etiq.pipeline.AccuracyMetricsIssuePipeline0099:Min threshold = 0.6, Max threshold = 1.0
INFO:etiq.pipeline.AccuracyMetricsIssuePipeline0099:Completed pipeline


In [16]:
issue_summary

,name,metric,measure,features,segments,total_issues_tested,issues_found,threshold
0,accuracy_below_threshold,<function accuracy at 0x7f192e1e49d0>,None,{},{},1,0,"[0.8, 1.0]"
1,true_pos_rate_below_threshold,<function true_pos_rate at 0x7f192e1e4af0>,None,{},{},1,0,"[0.6, 1.0]"
2,true_neg_rate_below_threshold,<function true_neg_rate at 0x7f192e1e4c10>,None,{},{},1,0,"[0.6, 1.0]"


# Bias Sources

Even if your training and test samples are very similar, the bias sources are related to properties of the training dataset, not the test dataset. 

To run bias sources even if you have an already trained model please use the TRAINING dataset instead.

You will need to change the train_test_valid in your config & then re-log your snapshots. You will still have the same model architecture from your already trained model, but if the scans show issues we recommend you clean your data and retrain the model. 

To be able to use bias sources you will have to log which features are categorical and which are numeric. 

You can do this in the config or in the notebook if easier. Commented out example shows you how to you do it in your notebook.



In [17]:
from etiq import Model
import json
# 
with etiq.etiq_config("./config_already_trained_bias_sources.json"):
    # Print out the config
    print(f"Etiq Configuration:\n{json.dumps(etiq.get_config(), indent=4)}")

    # Load the dataset
    dataset = etiq.BiasDatasetBuilder.datasets(training_features=train,
                                               validation_features=test)
    # Load the bias parameters
    bias_params = etiq.BiasDatasetBuilder.bias_params()

    # Create your already trained model and log it.
    model = Model(model_architecture=standard_model, model_fitted=model_fit)

    # Create the "Bias Sources Snapshot"
    snapshot = project.snapshots.create(name="Bias Sources (Testing) Snapshot",
                                        dataset=dataset,
                                        model=model,
                                        bias_params=bias_params)
    # Scan for sources of bias in the training data
    (bias_sources_segments_training,
     bias_sources_issues_training,
     bias_sources_issue_summary_training) = snapshot.scan_bias_sources()

Etiq Configuration:
{
    "dataset": {
        "label": "income",
        "bias_params": {
            "protected": "gender",
            "privileged": 1,
            "unprivileged": 0,
            "positive_outcome_label": 1,
            "negative_outcome_label": 0
        },
        "train_valid_test_splits": [
            1.0,
            0.01,
            0.0
        ],
        "remove_protected_from_features": true,
        "cat_col": [
            "workclass",
            "relationship",
            "occupation",
            "gender",
            "race",
            "native-country",
            "marital-status",
            "income",
            "education"
        ],
        "cont_col": [
            "age",
            "educational-num",
            "fnlwgt",
            "capital-gain",
            "capital-loss",
            "hours-per-week"
        ]
    },
    "scan_bias_sources": {
        "auto": true
    },
    "scan_accuracy_metrics": {
        "thresholds": {
          

In [18]:
bias_sources_issue_summary_training

,name,metric,measure,features,segments,total_issues_tested,issues_found,threshold
0,missing_sample,None,None,{},{98},99,1,"(0.0, 0.0)"
1,low_unpriv_sample,None,None,{},"{3, 17, 18, 19, 20, 21, 30, 31, 32, 34, 46, 49...",98,20,"(0.0, 0.8)"
2,low_priv_sample,None,None,{},"{1, 2, 67, 33, 5, 37, 38, 29}",98,8,"(0.0, 0.8)"
3,skewed_priv_sample,None,None,{},{},81,0,"(0.0, 0.2)"
4,skewed_unpriv_sample,None,None,{},"{96, 97, 40, 41, 42, 43, 13, 48, 17, 51, 83, 5...",98,17,"(0.0, 0.2)"
5,low_volume_group,None,None,{},{},99,0,"(781.36, inf)"
6,limited_features_issue,<function equal_opportunity at 0x7f192e1ee0d0>,None,{},"{4.0, 71.0, 42.0, 11.0, 12.0, 56.0, 95.0}",99,7,"(0.0, 0.2)"
7,proxy_issue,None,<function pointbiserial at 0x7f192e1deca0>,{},{},594,0,"(0.0, 1.0)"
8,proxy_issue,None,<function cramersv at 0x7f192e1deb80>,"{relationship, occupation, marital-status}","{3, 6, 8, 9, 13, 18, 19, 20, 23, 24, 26, 30, 3...",693,82,"(0.0, 1.0)"
9,correlation_issue,None,<function pointbiserial at 0x7f192e1deca0>,"{capital-gain, capital-loss, fnlwgt, age, educ...","{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",594,144,"(0.0, 1.0)"


In [19]:

with etiq.etiq_config("./config_already_trained_bias_sources2.json"):
    # Print out the config
    print(f"Etiq Configuration:\n{etiq.get_config()}")

    # Load the dataset
    dataset = etiq.BiasDatasetBuilder.datasets(training_features=test,
                                               validation_features=valid)
    # Load the bias parameters
    bias_params = etiq.BiasDatasetBuilder.bias_params()

    # Create your already trained model and log it.
    model = Model(model_architecture=standard_model, model_fitted=model_fit)

    # Create the "Bias Sources Snapshot"
    snapshot = project.snapshots.create(name="Bias Sources (Training) Snapshot",
                                        dataset=dataset,
                                        model=model,
                                        bias_params=bias_params)
    # Scan for sources of bias in the training data
    (bias_sources_segments_testing,
     bias_sources_issues_testing,
     bias_sources_issue_summary_testing) = snapshot.scan_bias_sources()

Etiq Configuration:
{'dataset': {'label': 'income', 'bias_params': {'protected': 'gender', 'privileged': 1, 'unprivileged': 0, 'positive_outcome_label': 1, 'negative_outcome_label': 0}, 'train_valid_test_splits': [1.0, 0.01, 0.0], 'remove_protected_from_features': True}, 'scan_bias_sources': {'auto': True}, 'scan_accuracy_metrics': {'thresholds': {'accuracy': [0.8, 1.0], 'true_pos_rate': [0.6, 1.0], 'true_neg_rate': [0.6, 1.0]}}, 'scan_bias_metrics': {'thresholds': {'equal_opportunity': [0.0, 0.2], 'demographic_parity': [0.0, 0.2], 'equal_odds_tnr': [0.0, 0.2], 'individual_fairness': [0.0, 0.8], 'equal_odds_tpr': [0.0, 0.2]}}, 'scan_leakage': {'leakage_threshold': 0.85}}
INFO:etiq.charting:Created histogram summary of data (14 fields)
INFO:etiq.pipeline.DataPipeline0020:Starting pipeline
INFO:etiq.pipeline.DataPipeline0020:Computed metrics for the initial dataset
INFO:etiq.pipeline.DataPipeline0020:Completed pipeline
Running auto bias sources pipeline
INFO:etiq.pipeline.DebiasPipeline0

In [20]:
bias_sources_segments_testing


,name,business_rule,mask,tags,is_global,number_of_samples
0,0,`relationship` == 3 and `fnlwgt` <= 308205.865...,"[False, False, False, False, False, False, Fal...",{},False,0
1,1,`relationship` == 3 and `fnlwgt` > 308205.8652...,"[False, False, False, False, False, False, Fal...",{},False,0
2,2,`relationship` == 1 and `fnlwgt` <= 51917.3926...,"[False, False, False, False, False, False, Fal...",{},False,0
3,3,`relationship` == 1 and `fnlwgt` > 51917.39266...,"[False, False, False, False, False, False, Tru...",{},False,0
4,4,`relationship` == 1 and `fnlwgt` > 51917.39266...,"[False, False, False, False, False, False, Fal...",{},False,0
...,...,...,...,...,...,...
110,110,`relationship` == 1 and `race` == 4 and `fnlwg...,"[False, False, False, False, False, False, Fal...",{},False,0
111,111,`relationship` == 1 and `race` == 2,"[False, False, False, False, False, False, Fal...",{},False,0
112,112,`relationship` == 4 and `education` > 8.0,"[False, False, False, False, False, False, Fal...",{},False,0
113,113,`relationship` == 0,"[False, False, False, True, False, True, False...",{},False,0
